### **First Stage Sentence Retrieval using SBERT**

In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


Matplotlib created a temporary cache directory at C:\Users\BILLZH~1\AppData\Local\Temp\matplotlib-11uofcfb because the default path (C:\Users\Bill Zhu\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
import json

# Load datasets
path = 'data/'

with open(path+"train-claims.json", "r") as f:
    train_claims = json.load(f)

train_df = pd.DataFrame(train_claims).transpose()

with open(path+"dev-claims.json", "r") as f:
    dev_claims = json.load(f)
dev_df = pd.DataFrame(dev_claims).transpose()

with open(path+"evidence.json", "r") as f:
    evidence = json.load(f)
evidence_df = pd.DataFrame(list(evidence.items()), columns=["key", "value"])

In [ ]:
# Load model
model1 = SentenceTransformer('sbert_model2')
model2 = SentenceTransformer('sbert_model3')


In [4]:
embeddings1 = model1.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')
embeddings2 = model2.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

In [5]:
claim_embeddings1 = model1.encode(train_df['claim_text'].tolist(), show_progress_bar=True, device='cuda')
claim_embeddings2 = model2.encode(train_df['claim_text'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

In [6]:
print(claim_embeddings2) # embeddings understand what they mean..

[[ 0.01187707 -0.02390274 -0.0748487  ...  0.05677928  0.11237261
  -0.03542037]
 [ 0.07561179 -0.01496751 -0.01832518 ... -0.10404789  0.05262688
   0.05971221]
 [-0.01100291 -0.03878411 -0.04935465 ...  0.03594766 -0.02314097
  -0.06052085]
 ...
 [ 0.02827161  0.04342072 -0.07901847 ... -0.00902957 -0.00489259
  -0.00168524]
 [-0.02562358  0.04905974 -0.00427667 ... -0.05533928 -0.01342698
   0.06300803]
 [-0.00428972 -0.01722773 -0.13271657 ... -0.05778776  0.09227562
  -0.01590666]]


In [7]:
claim_combined = (claim_embeddings1 + claim_embeddings2) / 2 
evidence_combined = (embeddings1 + embeddings2) / 2


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(claim_combined , evidence_combined)

In [9]:
import numpy as np
# Get top 10000 evidence for each claim
top_k = 25
top_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]  # sort descending, get top 100 indices

In [10]:
# Create new column with top evidence texts for each claim
train_df["top_100_evidence"] = [
    evidence_df.iloc[indices]["key"].tolist() for indices in top_indices
]

In [11]:
total_matches = 0
n = 0 
for i in range(len(train_df)):
    k = 0
    evidence_list = train_df["evidences"].iloc[i]
    top_100_list = train_df["top_100_evidence"].iloc[i]

    for ev in evidence_list:
        if ev in top_100_list:
            k += 1
    print(f"Found {k} matches for gold standard {len(evidence_list)}")
    n += len(evidence_list)
    total_matches += k

print(f"Total matches: {total_matches}/{n}")

Found 2 matches for gold standard 3
Found 0 matches for gold standard 2
Found 1 matches for gold standard 2
Found 1 matches for gold standard 5
Found 3 matches for gold standard 5
Found 0 matches for gold standard 5
Found 2 matches for gold standard 2
Found 0 matches for gold standard 3
Found 2 matches for gold standard 5
Found 1 matches for gold standard 1
Found 1 matches for gold standard 1
Found 4 matches for gold standard 5
Found 0 matches for gold standard 5
Found 4 matches for gold standard 5
Found 1 matches for gold standard 1
Found 0 matches for gold standard 1
Found 2 matches for gold standard 3
Found 2 matches for gold standard 5
Found 0 matches for gold standard 1
Found 4 matches for gold standard 4
Found 1 matches for gold standard 2
Found 2 matches for gold standard 2
Found 3 matches for gold standard 4
Found 2 matches for gold standard 3
Found 0 matches for gold standard 4
Found 2 matches for gold standard 2
Found 0 matches for gold standard 1
Found 5 matches for gold sta

In [ ]:
### cross encoder
from datasets import Dataset

claims = []
evidence_texts = []
labels = []

evidence_map = dict(evidence)
evidence_ids = list(evidence_map.keys())

for row in train_df.itertuples():
    claim_text = row.claim_text
    positive_ids = set(row.evidences)
    negative_ids = set(row.top_100_evidence) - positive_ids

    # === Add all golden (positive) evidences ===
    for eid in positive_ids:
        if eid in evidence_map:
            claims.append(claim_text)
            evidence_texts.append(evidence_map[eid])
            labels.append(1)
    
    for eid in negative_ids:
        if eid in evidence_map:
            claims.append(claim_text)
            evidence_texts.append(evidence_map[eid])
            labels.append(-1)

# onvert to HuggingFace Dataset
data = {
    "claim": claims,
    "evidence": evidence_texts,
    "label": labels
}

dataset = Dataset.from_dict(data)

# Confirm
print(dataset)
print(dataset[0:5])

In [ ]:
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.losses import MultipleNegativesRankingLoss

model = CrossEncoder("cross-encoder/stsb-roberta-base")
loss = MultipleNegativesRankingLoss(model)

In [ ]:
import logging
import traceback

from datasets import load_dataset

from sentence_transformers.cross_encoder import (
    CrossEncoder,
    CrossEncoderModelCardData,
    CrossEncoderTrainer,
    CrossEncoderTrainingArguments,
)

split_dataset = dataset.train_test_split(test_size=0.1, seed=42069)
train_ds = split_dataset["train"]
eval_ds = split_dataset["test"]

args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir=f"output/",
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    logging_first_step=True,
    run_name='cross',  # Will be used in W&B if `wandb` is installed
    seed=12,
)

trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss=loss,
)
trainer.train()

trainer.save_model("cross_encoder_model")

In [12]:
# from sentence_transformers import CrossEncoder, InputExample
# from torch.utils.data import DataLoader
# from sklearn.model_selection import train_test_split

# # Step 1: Prepare InputExamples
# train_examples = []
# for claim, top_evidence_list, gold_list in zip(train_df['claim_text'], train_df['top_100_evidence'], train_df['evidences']):
#     for ev in top_evidence_list:
#         label = 1 if ev in gold_list else 0
#         train_examples.append(InputExample(texts=[claim, ev], label=label))

# # Step 2: Train/Dev Split
# train_data, dev_data = train_test_split(train_examples, test_size=0.1, random_state=42)

# # Step 3: CrossEncoder setup
# model = CrossEncoder("cross-encoder/stsb-roberta-base")

# from sentence_transformers.cross_encoder.evaluation import CrossEncoderClassificationEvaluator

# #evaluator = CrossEncoderClassificationEvaluator.from_input_examples(dev_data, name="dev-eval")


# # Step 4: Fine-tune
# model.fit(
#     train_dataloader=DataLoader(train_data, shuffle=True, batch_size=16),
#     epochs=2,
#     warmup_steps=100,
#     output_path="cross-encoder-finetuned"
# )

# # save the model
# model.save("cross-encoder-finetuned")

In [13]:
# from datasets import Dataset
# from sklearn.model_selection import train_test_split

# # Prepare raw data as a list of dictionaries
# pairs = []
# for claim, top_evidence_list, gold_list in zip(train_df['claim_text'], train_df['top_100_evidence'], train_df['evidences']):
#     for ev in top_evidence_list:
#         label = 1 if ev in gold_list else 0
#         pairs.append({'text_pair': [claim, ev], 'label': label})

In [14]:
# train_data, dev_data = train_test_split(pairs, test_size=0.1, random_state=42)

# train_dataset = Dataset.from_list(train_data)
# dev_dataset = Dataset.from_list(dev_data)

In [15]:
# ### loss function
# from datasets import load_dataset
# from sentence_transformers import CrossEncoder
# from sentence_transformers.cross_encoder.losses import MultipleNegativesRankingLoss

# # Load a model to train/finetune
# model = CrossEncoder("cross-encoder/stsb-roberta-base")

# # Initialize the MultipleNegativesRankingLoss
# # This loss requires pairs of related texts or triplets
# loss = MultipleNegativesRankingLoss(model)


In [16]:
# from sentence_transformers.cross_encoder import CrossEncoderTrainingArguments

# args = CrossEncoderTrainingArguments(
#     # Required parameter:
#     output_dir="models/xlm-roberta-base",
#     # Optional training parameters:
#     num_train_epochs=1,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     learning_rate=2e-5,
#     warmup_ratio=0.1,
#     fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
#     bf16=False,  # Set to True if you have a GPU that supports BF16
#     # Optional tracking/debugging parameters:
#     eval_strategy="steps",
#     eval_steps=100,
#     save_strategy="steps",
#     save_steps=100,
#     save_total_limit=2,
#     logging_steps=100,
#     run_name="xlm-roberta-base",  # Will be used in W&B if `wandb` is installed
# )

In [17]:
# from sentence_transformers import CrossEncoder
# from datasets import load_dataset
# from sentence_transformers.cross_encoder import (
#     CrossEncoder,
#     CrossEncoderModelCardData,
#     CrossEncoderTrainer,
#     CrossEncoderTrainingArguments,
# )
# from sentence_transformers.losses import MultipleNegativesRankingLoss
# from sentence_transformers.training_args import BatchSamplers
# from sentence_transformers.evaluation import TripletEvaluator
# from sentence_transformers.cross_encoder.evaluation import CrossEncoderNanoBEIREvaluator
# from sentence_transformers.cross_encoder.losses import CachedMultipleNegativesRankingLoss

# train_batch_size = 64

# evaluator = CrossEncoderNanoBEIREvaluator(
#     dataset_names=["msmarco", "nfcorpus", "nq"],
#     batch_size=train_batch_size,
# )

# trainer = CrossEncoderTrainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=dev_dataset,
#     loss=loss,
#     evaluator=evaluator,
# )
# trainer.train()